# Задание 8
### На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара

In [3]:
import pandas as pd
import numpy as np

import nltk
from nltk import word_tokenize
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import time

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Flatten, Dropout, MaxPooling1D, Embedding,\
        GlobalMaxPool1D, BatchNormalization, Bidirectional, SimpleRNN, GRU, Masking
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import utils

In [4]:
df_train = pd.read_csv('data/train.csv', index_col='id')
df_val = pd.read_csv('data/val.csv', index_col='id')
# test.csv без классов, здесь не пригодиться

In [5]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)

In [6]:
# считаем количество уникальных слов в отзыве (максимум, в среднем)
df_train['text'].apply(lambda x: len(np.unique(str(x).split(' ')))).max(),\
df_train['text'].apply(lambda x: len(np.unique(str(x).split(' ')))).mean()

(24, 7.0138537585346095)

In [7]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values

In [8]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])


In [9]:
%%time
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_val, maxlen=training_length)

Wall time: 609 ms


In [10]:
# y_train = utils.to_categorical(df_train['class'], num_classes=num_classes)
# y_test = utils.to_categorical(df_val['class'], num_classes=num_classes)

In [11]:
y_train = df_train['class'].values
y_test = df_val['class'].values

In [12]:
word_count, training_length

(258108, 27)

### 1. построить свёрточные архитектуры

In [14]:
model_cnn = Sequential([
    Embedding(input_dim=word_count, output_dim=64, input_length=training_length),
    Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
    GlobalMaxPool1D(),
    Dense(units=32, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

model_cnn.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [15]:
batch_size = 512
epochs = 5

In [16]:
%%time
history = model_cnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

Epoch 1/5
355/355 [==============================] - 80s 227ms/step - loss: 0.5374 - accuracy: 0.7179 - val_loss: 0.5023 - val_accuracy: 0.7531
Epoch 2/5
355/355 [==============================] - 79s 223ms/step - loss: 0.2750 - accuracy: 0.8865 - val_loss: 0.5935 - val_accuracy: 0.7400
Epoch 3/5
355/355 [==============================] - 83s 234ms/step - loss: 0.0916 - accuracy: 0.9668 - val_loss: 0.8172 - val_accuracy: 0.7246
Epoch 4/5
355/355 [==============================] - 81s 228ms/step - loss: 0.0336 - accuracy: 0.9891 - val_loss: 0.9860 - val_accuracy: 0.7221
Epoch 5/5
355/355 [==============================] - 81s 228ms/step - loss: 0.0131 - accuracy: 0.9963 - val_loss: 1.2449 - val_accuracy: 0.7211
Wall time: 6min 47s


In [17]:
history.history['val_accuracy'][-1]

0.7211127281188965

In [18]:
model_cnn = Sequential([
    Embedding(input_dim=word_count, output_dim=32, input_length=training_length),
    Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
    Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'),
    GlobalMaxPool1D(),
    Dense(units=32, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

model_cnn.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [19]:
%%time
history = model_cnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
history.history['val_accuracy'][-1]

Epoch 1/5
355/355 [==============================] - 42s 117ms/step - loss: 0.5380 - accuracy: 0.7143 - val_loss: 0.5104 - val_accuracy: 0.7450
Epoch 2/5
355/355 [==============================] - 41s 117ms/step - loss: 0.2695 - accuracy: 0.8884 - val_loss: 0.6011 - val_accuracy: 0.7371
Epoch 3/5
355/355 [==============================] - 41s 117ms/step - loss: 0.0961 - accuracy: 0.9655 - val_loss: 0.7904 - val_accuracy: 0.7279
Epoch 4/5
355/355 [==============================] - 42s 117ms/step - loss: 0.0430 - accuracy: 0.9852 - val_loss: 0.9811 - val_accuracy: 0.7206
Epoch 5/5
355/355 [==============================] - 42s 117ms/step - loss: 0.0200 - accuracy: 0.9935 - val_loss: 1.2219 - val_accuracy: 0.7160
Wall time: 3min 28s


0.7160428762435913

In [20]:
model_cnn = Sequential([
    Embedding(input_dim=word_count, output_dim=32, input_length=training_length),
    Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
    BatchNormalization(),
    Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'),
    GlobalMaxPool1D(),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

model_cnn.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [21]:
%%time
history = model_cnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
history.history['val_accuracy'][-1]

Epoch 1/5
355/355 [==============================] - 42s 118ms/step - loss: 0.5429 - accuracy: 0.7110 - val_loss: 0.6087 - val_accuracy: 0.7455
Epoch 2/5
355/355 [==============================] - 42s 119ms/step - loss: 0.2791 - accuracy: 0.8831 - val_loss: 0.5619 - val_accuracy: 0.7324
Epoch 3/5
355/355 [==============================] - 42s 119ms/step - loss: 0.0874 - accuracy: 0.9684 - val_loss: 0.8286 - val_accuracy: 0.7328
Epoch 4/5
355/355 [==============================] - 42s 119ms/step - loss: 0.0316 - accuracy: 0.9893 - val_loss: 1.2691 - val_accuracy: 0.7179
Epoch 5/5
355/355 [==============================] - 42s 119ms/step - loss: 0.0150 - accuracy: 0.9949 - val_loss: 1.4042 - val_accuracy: 0.7253
Wall time: 3min 32s


0.7252568006515503

###  2. построить различные архитектуры с RNN

In [22]:
model_rnn = Sequential([
    Embedding(input_dim=word_count, output_dim=32, input_length=training_length, mask_zero=True),
    Masking(mask_value=0.0),    
    LSTM(32),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

model_rnn.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [23]:
%%time
history = model_rnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
history.history['val_accuracy'][-1]

Epoch 1/5
355/355 [==============================] - 42s 118ms/step - loss: 0.5492 - accuracy: 0.7117 - val_loss: 0.5094 - val_accuracy: 0.7432
Epoch 2/5
355/355 [==============================] - 41s 116ms/step - loss: 0.3238 - accuracy: 0.8636 - val_loss: 0.5828 - val_accuracy: 0.7363
Epoch 3/5
355/355 [==============================] - 41s 117ms/step - loss: 0.1628 - accuracy: 0.9382 - val_loss: 0.7281 - val_accuracy: 0.7280
Epoch 4/5
355/355 [==============================] - 42s 118ms/step - loss: 0.1026 - accuracy: 0.9626 - val_loss: 0.8945 - val_accuracy: 0.7237
Epoch 5/5
355/355 [==============================] - 42s 119ms/step - loss: 0.0755 - accuracy: 0.9720 - val_loss: 0.9665 - val_accuracy: 0.7186
Wall time: 3min 33s


0.7185998558998108

In [26]:
model_rnn = Sequential([
    Embedding(input_dim=word_count, output_dim=32, input_length=training_length, mask_zero=True),
    Masking(mask_value=0.0),    
    SimpleRNN(64),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

model_rnn.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [27]:
%%time
history = model_rnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
history.history['val_accuracy'][-1]

Epoch 1/5
355/355 [==============================] - 43s 121ms/step - loss: 0.5526 - accuracy: 0.7051 - val_loss: 0.5199 - val_accuracy: 0.7405
Epoch 2/5
355/355 [==============================] - 43s 121ms/step - loss: 0.2759 - accuracy: 0.8875 - val_loss: 0.5935 - val_accuracy: 0.7323
Epoch 3/5
355/355 [==============================] - 43s 120ms/step - loss: 0.0814 - accuracy: 0.9711 - val_loss: 0.8009 - val_accuracy: 0.7209
Epoch 4/5
355/355 [==============================] - 43s 120ms/step - loss: 0.0344 - accuracy: 0.9884 - val_loss: 0.9849 - val_accuracy: 0.7184
Epoch 5/5
355/355 [==============================] - 43s 122ms/step - loss: 0.0194 - accuracy: 0.9937 - val_loss: 1.1319 - val_accuracy: 0.7159
Wall time: 3min 36s


0.7159106135368347

In [28]:
model_rnn = Sequential([
    Embedding(input_dim=word_count, output_dim=32, input_length=training_length, mask_zero=True),
    Masking(mask_value=0.0),    
    GRU(64, recurrent_dropout=0.2),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

model_rnn.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [29]:
%%time
history = model_rnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
history.history['val_accuracy'][-1]

Epoch 1/5
355/355 [==============================] - 56s 157ms/step - loss: 0.5448 - accuracy: 0.7159 - val_loss: 0.5049 - val_accuracy: 0.7473
Epoch 2/5
355/355 [==============================] - 55s 156ms/step - loss: 0.3102 - accuracy: 0.8707 - val_loss: 0.5738 - val_accuracy: 0.7404
Epoch 3/5
355/355 [==============================] - 55s 154ms/step - loss: 0.1540 - accuracy: 0.9417 - val_loss: 0.7215 - val_accuracy: 0.7320
Epoch 4/5
355/355 [==============================] - 58s 162ms/step - loss: 0.0987 - accuracy: 0.9640 - val_loss: 0.9500 - val_accuracy: 0.7174
Epoch 5/5
355/355 [==============================] - 55s 155ms/step - loss: 0.0706 - accuracy: 0.9739 - val_loss: 0.9942 - val_accuracy: 0.7212
Wall time: 4min 41s


0.7211568355560303

In [36]:
model_rnn = Sequential([
    Embedding(input_dim=word_count, output_dim=32, input_length=training_length, mask_zero=True),
    Masking(mask_value=0.0),    
    Bidirectional(LSTM(64, recurrent_dropout=0.2)),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

model_rnn.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [37]:
%%time
history = model_rnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
history.history['val_accuracy'][-1]

Epoch 1/5
355/355 [==============================] - 87s 245ms/step - loss: 0.5460 - accuracy: 0.7135 - val_loss: 0.4982 - val_accuracy: 0.7511
Epoch 2/5
355/355 [==============================] - 86s 243ms/step - loss: 0.3186 - accuracy: 0.8680 - val_loss: 0.5890 - val_accuracy: 0.7392
Epoch 3/5
355/355 [==============================] - 86s 241ms/step - loss: 0.1613 - accuracy: 0.9390 - val_loss: 0.7621 - val_accuracy: 0.7201
Epoch 4/5
355/355 [==============================] - 86s 242ms/step - loss: 0.1042 - accuracy: 0.9620 - val_loss: 0.8563 - val_accuracy: 0.7230
Epoch 5/5
355/355 [==============================] - 87s 245ms/step - loss: 0.0777 - accuracy: 0.9716 - val_loss: 0.9420 - val_accuracy: 0.7184
Wall time: 7min 17s


0.7184234857559204

### 3. попробовать использовать совместно CNN и RNN

In [38]:
model_mix = Sequential([
    Embedding(input_dim=word_count, output_dim=64, input_length=training_length),
    Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
    BatchNormalization(),
    Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'),
    Bidirectional(LSTM(64, recurrent_dropout=0.2)),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

model_mix.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [39]:
%%time
history = model_mix.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
history.history['val_accuracy'][-1]

Epoch 1/5
355/355 [==============================] - 111s 312ms/step - loss: 0.5272 - accuracy: 0.7259 - val_loss: 0.5730 - val_accuracy: 0.7466
Epoch 2/5
355/355 [==============================] - 112s 314ms/step - loss: 0.2373 - accuracy: 0.9036 - val_loss: 0.5757 - val_accuracy: 0.7366
Epoch 3/5
355/355 [==============================] - 111s 313ms/step - loss: 0.0679 - accuracy: 0.9757 - val_loss: 0.8400 - val_accuracy: 0.7321
Epoch 4/5
355/355 [==============================] - 112s 316ms/step - loss: 0.0230 - accuracy: 0.9925 - val_loss: 1.2007 - val_accuracy: 0.7214
Epoch 5/5
355/355 [==============================] - 112s 317ms/step - loss: 0.0116 - accuracy: 0.9962 - val_loss: 1.5538 - val_accuracy: 0.7294
Wall time: 9min 24s


0.7294008731842041

### 4. сделать выводы что получилось лучше

Все подходы показывают соизмеримый результат, около ~0.72. По крайней мере разница не настолько существенна, чтобы какой-то из методов отправлять на свалку. <br>
Комбинирование методов усложняет модели, и соответственно время работы.